<a href="https://colab.research.google.com/github/manehharutyunyan/ComplexNumbers/blob/master/YSU_NLP_Text_Classification_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # The package `datasets` provides a big number of datasets for NLP tasks.
# No need to manually download and read the files.
!pip install datasets
# TQDM is used to track the progress of your code.
# Wrap long-running loops
!pip install tqdm

     |████████████████████████████████| 270 kB 20.3 MB/s 
     |████████████████████████████████| 1.3 MB 29.6 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 123 kB 33.4 MB/s 
     |████████████████████████████████| 243 kB 72.0 MB/s 
     |████████████████████████████████| 294 kB 61.4 MB/s 
     |████████████████████████████████| 142 kB 81.2 MB/s 


In [ ]:
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
# `load_dataset()` automatically pulls, caches and reads the dataset you want
dataset = load_dataset('amazon_reviews_multi', 'en')

Downloading:   0%|          | 0.00/2.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.62k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/82.0M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [ ]:
# As you can see, three splits are provided.
# Please DO NOT USE THE TEST SET for any purposes. DON'T EVEN TOUCH IT.
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [ ]:
# You can pick a sample from dataset
dataset['train'][1]

{'language': 'en',
 'product_category': 'home_improvement',
 'product_id': 'product_en_0440378',
 'review_body': 'the cabinet dot were all detached from backing... got me',
 'review_id': 'en_0690095',
 'review_title': 'Not use able',
 'reviewer_id': 'reviewer_en_0133349',
 'stars': 1}

In [ ]:
# It's easy to verify that `stars` are in the range [1, 5]
set(dataset['train']['stars'])

{1, 2, 3, 4, 5}

In [ ]:
# The homework is to implement a dictionary-based text classification.
# In short, read all the dataset and try to find the most positive and most negative words.
# 
# A little help for you will be this mini-map:
# 1. First, define your tokenization. Spacy will work ok.
# 2. Loop over the dataset, collect the set of all the words / tokens
#    Please use only `review_body` as the input, and `stars` as the label. 
# 3. Loop over the dataset, collect the number of times the token occurred in (*) the dataset
#                                                                             (*) the positive samples
#                                                                             (*) the negative samples
#    Consider this task as binary classification task.
#    We define `positive samples` as ones with either `sample['stars'] == 4` or `sample['stars'] == 5`
#    We define `negative samples` as ones with either `sample['stars'] == 1` or `sample['stars'] == 2`
#    We ignore all the reviews where `sample['stars'] == 3`. (Neutral reviews)
# 
# In the test mode, then input text is given, look at all the tokens in it.
# Count the average `positiveness` score of all the tokens.
#   If it's a positive => classify the text as positive, otherwise - negative.
# Let's define the `positiveness` score of a token as
#   (stat_positives[token] - stat_negatives[token]) / stat_total[token]
# Where stat_positives[token] is the number of times the `token` appeared in the positive examples.
# stat_total[token] is the total number of times the `token` was seen in the dataset, no matter the `stars` rating.

In [ ]:
#pip install -U spacy


In [ ]:
#1
from spacy.lang.en import English
nlp = English()

def spacy_tokenize(sentence):
    return [
        token.text for token in nlp(sentence)
    ]

token_list = []
for sample in dataset['train']:
  token_list.extend(spacy_tokenize(sample['review_body']))

In [ ]:
token_list = list(set(token_list))

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
datasetLen = len(set(token_list))

W_dictionary = {}
for t in set(token_list):
  W_dictionary[t] = [0, 0]

for sample in dataset['train']:
  if(sample['stars'] == 4 or sample['stars'] == 5):
    for word in spacy_tokenize(sample['review_body']):
      W_dictionary[word][0] += 1
  elif(sample['stars'] == 2 or sample['stars'] == 1):
    for word in spacy_tokenize(sample['review_body']):
      W_dictionary[word][1] += 1


In [ ]:
W = pd.DataFrame.from_dict(W_dictionary).T
W.columns = ['Positive', 'Negative']

In [ ]:
W['Total score'] = W['Positive'] + W['Negative']

In [ ]:
preds_sent = []
tokens = set(token_list)

for sample in tqdm(dataset['test']):
  tokens = spacy_tokenize(sample['review_body'])
  predictions = []

  for token in tokens:
    one_hot_sentence = np.zeros(len(token_list))
    try:
      one_hot_sentence[token_list.index(token)] = 1
    except:
      continue
    
    output = one_hot_sentence @ W.values
    prediction = (output[0] - output[1]) / output[2]
    predictions.append(prediction)
  preds_sent.append(np.mean(predictions))

  0%|          | 16/5000 [00:02<09:34,  8.68it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
 53%|█████▎    | 2645/5000 [06:12<08:52,  4.42it/s]

In [ ]:
preds = np.array(preds)
len(preds[preds>0])

382

In [ ]:
stars = []
for sample in dataset['test']:
  stars.append(sample['stars'])

In [ ]:
stars = np.array(stars)
len(stars[stars>=4])

2000